In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
articles=pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv')
sub_pd=pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
transaction=pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv',usecols=['t_dat','customer_id','article_id'],dtype={'article_id': np.string_})

# Data pre-processing

In [ ]:
transaction['customer_id'].unique()
dif=[]

ss=sub_pd['customer_id'].unique().tolist()[:10]
tt=transaction['customer_id'].unique().tolist()[:10]
for s in ss:
    if s not in tt:
        print(s)
        dif.append(s)
        
del ss
del tt
print("number of who have not bought before",len(dif))

In [ ]:
top12=transaction.groupby('article_id')['customer_id'].nunique().sort_values(ascending=False).head(12).index.astype(int)
top12_forsub=''
for t in top12:
    top12_forsub=top12_forsub+' '+str(t)
del top12
top12_forsub=top12_forsub[1:]
top12_forsub

We recommend this items who didnt buy before.

In [ ]:
articles.index=articles['article_id']
articles=articles.loc[transaction['article_id'].unique().astype(int).tolist()]
articles.head()
articles=articles.drop(['article_id'], axis=1)

articles=articles.drop(['product_code', 'prod_name','product_group_name','department_no','product_type_no','index_name','graphical_appearance_no','graphical_appearance_name','colour_group_code','index_code','index_group_no','section_no','garment_group_no','detail_desc','perceived_colour_value_id', 'perceived_colour_value_name','perceived_colour_master_id', 'perceived_colour_master_name'],axis=1)    
articles['article_id']=articles.index
articles.head()

In [ ]:
#dummify categorical features
article_data = pd.get_dummies(articles, columns = ['product_type_name'])
del articles
article_data = article_data.reset_index(drop=True)
article_data = article_data.sort_values('article_id').reset_index().drop('index', axis=1)
article_data.head(5)

In [ ]:
from scipy.sparse import csr_matrix
article_data=article_data.drop(['colour_group_name','department_name','index_group_name','section_name','garment_group_name'],axis=1)    
articles_csr = csr_matrix(article_data.drop('article_id', axis=1).values)
del article_data
articles_csr

In [ ]:
transaction['t_dat']=pd.to_datetime(transaction['t_dat'])
transaction.index=transaction['t_dat']
transaction=transaction['2018-09-20':'2018-09-23']
transaction.index=transaction['customer_id']
transaction.drop(['customer_id','t_dat'],axis=1,inplace=True)
transaction_dm=pd.get_dummies(transaction,prefix='',prefix_sep='')
del transaction

But we can do this for a few days because the data is huge. 16gb memory and time is not enough for all data.

In [ ]:
from scipy.sparse import csr_matrix
from tqdm import tqdm
transaction_csr = csr_matrix(transaction_dm.values)
transaction_csr

In [ ]:
user_id = list(transaction_dm.index)
user_dict = {}
counter = 0 
for i in user_id:
    user_dict[i] = counter
    counter += 1

# Modelling

In [ ]:
from lightfm import LightFM
model = LightFM(loss='warp',
                random_state=2016,
                learning_rate=0.90,
                no_components=150,
                user_alpha=0.000005)

model = model.fit(transaction_csr,
                  epochs=100,
                  num_threads=16, verbose=False)

# Submission

In [ ]:
def sample_recommendation_user(model,interactions,sub_pd,user_dict,threshold = 0,nrec_items = 12, show = True):
    j=-1
    for user_id in tqdm(sub_pd.customer_id):
        j=j+1
        n_users, n_items = interactions.shape
        if user_id in user_dict:
            user_x = user_dict[user_id]
            scores = pd.Series(model.predict(user_x,np.arange(n_items), item_features=articles_csr))
            scores.index = interactions.columns
            scores = list(pd.Series(scores.sort_values(ascending=False).index)) 
            known_items = list(pd.Series(interactions.loc[user_id,:] \
                                         [interactions.loc[user_id,:] > threshold].index).sort_values(ascending=False))
            scores = [x for x in scores if x not in known_items]
            return_score_list = scores[0:nrec_items]
            a=''
            for i in return_score_list:
                a=a+' '+str(i)
                a=a[1:]
            sub_pd['prediction'][j]=a
        else:
            sub_pd['prediction'][j]=top12_forsub
    return sub_pd

In [ ]:
sub_pd=sample_recommendation_user(model, transaction_dm,sub_pd, user_dict)
sub_pd.to_csv("submission.csv", index=False)